In [1]:
import json
from pyld import jsonld
import utils

In [114]:
with open('../examples_conductivity/single_result.json') as f:
    data = json.load(f)


with open("./mapping.json") as f:
    mapping = json.load(f)

In [101]:
context = {
  "@context": {
    "ctime": "http://example.com/vocab/ctime",
    "formulation": "http://example.com/vocab/formulation",
    "chemical": "http://example.com/vocab/chemical",
    "SMILES": "http://example.com/vocab/SMILES",
    "InChIKey": "http://example.com/vocab/InChIKey",
    "fraction": "http://example.com/vocab/fraction",
    "fraction_type": "http://example.com/vocab/fraction_type",
    "internal_reference": "http://example.com/vocab/internal_reference",
    "conductivity": "http://example.com/vocab/conductivity",
    "values": "http://example.com/vocab/values",
    "temperature": "http://example.com/vocab/temperature",
    "quantity": "http://example.com/vocab/quantity",
    "method": "http://example.com/vocab/method",
    "parameters": "http://example.com/vocab/parameters",
    "molecular_dynamics": "http://example.com/vocab/molecular_dynamics",
  }
}

jsonld_frame = {
  "@context": {
    "ctime": "http://example.com/vocab/ctime",
    "conductivity": "http://example.com/vocab/conductivity",
    "values": "http://example.com/vocab/values",
    "temperature": "http://example.com/vocab/temperature",
    "formulation": "http://example.com/vocab/formulation",
    "chemical": "http://example.com/vocab/chemical",
    "SMILES": "http://example.com/vocab/SMILES",
    "InChIKey": "http://example.com/vocab/InChIKey"
  },
  "@explicit": True,
  "@requireAll": True,
  "ctime": {},
  "result":{
      "@explicit": True,
      "formulation":{
          "chemical": {
            "SMILES": {},
            "InChIKey": {}
          }
      }
  }
  }


In [ ]:
mapping={
    "ctime": "http://example.com/vocab/ctime",
    "http://example.com/vocab/formulation",
    "chemical": "http://example.com/vocab/chemical",
    "SMILES": "http://example.com/vocab/SMILES",
    "InChIKey": "http://example.com/vocab/InChIKey",
    "fraction": "http://example.com/vocab/fraction",
    "fraction_type": "http://example.com/vocab/fraction_type",
    "internal_reference": "http://example.com/vocab/internal_reference",
    "conductivity": "http://example.com/vocab/conductivity",
    "values": "http://example.com/vocab/values",
    "temperature": "http://example.com/vocab/temperature",
    "quantity": "http://example.com/vocab/quantity",
    "method": "http://example.com/vocab/method",
    "parameters": "http://example.com/vocab/parameters",
    "molecular_dynamics": "http://example.com/vocab/molecular_dynamics",
}

In [133]:
def flatten_json(json_doc):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '/')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '/')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(json_doc)

    return out



def skip_levels(json_doc, to_skip):

    updated_y = {}
    for key, value in json_doc.items():
        
        new_key = "/".join([json_level for json_level in key.split("/") if json_level not in to_skip])
        updated_y[new_key]=value
        
    return updated_y



def nest_json(flat_dict):
    nested_dict = {}
    
    for key, value in flat_dict.items():
        keys = key.split("/")
        current_dict = nested_dict

        for k in keys[:-1]:
            current_dict = current_dict.setdefault(k, {})

        last_key = keys[-1]
        if last_key.isdigit():
            current_dict = current_dict.setdefault(keys[-2], [])
            current_dict.append(value)
        else:
            current_dict[last_key] = value

    return nested_dict




In [134]:
flattened_json_doc = flatten_json(data)
simplified_flattened_json_doc = skip_levels(flattened_json_doc, to_skip=["result", "data", "run_info"])
renested_json_doc = nest_json(simplified_flattened_json_doc)
renested_json_doc

{'uuid': '5362797a-e5d9-45ec-90a2-4f79be84d683',
 'ctime': '2023-07-26T11:22:20',
 'status': 'original',
 'formulation': {'0': {'chemical': {'SMILES': '[Li+].F[P-](F)(F)(F)(F)F',
    'InChIKey': 'AXPLOJNSKRXQPA-UHFFFAOYSA-N'},
   'fraction': 0.05,
   'fraction_type': 'molar fraction'},
  '1': {'chemical': {'SMILES': 'C1COC(=O)O1',
    'InChIKey': 'KMTRUDSVKNLOMY-UHFFFAOYSA-N'},
   'fraction': 0.45,
   'fraction_type': 'molar fraction'},
  '2': {'chemical': {'SMILES': 'CC1COC(=O)O1',
    'InChIKey': 'RUOJZAUFBMNUDX-UHFFFAOYSA-N'},
   'fraction': 0.5,
   'fraction_type': 'molar fraction'}},
 'internal_reference': 'a11c15de-6d14-424e-8dc6-32f8398bfd51',
 'conductivity': {'values': {'values': [0.008328539145648056]},
  'temperature': 298,
  'meta': {'success': True, 'rating': 2}},
 'quantity': 'conductivity',
 'method': {'method': ['molecular_dynamics']},
 'parameters': {'molecular_dynamics': {'formulation': {'0': {'chemical': {'SMILES': '[Li+].F[P-](F)(F)(F)(F)F',
      'InChIKey': 'AXPLO

In [111]:
"/".join([key for key in 'result/data/run_info/formulation/0/chemical/SMILES'.split("/") if key not in ["result", "data", "run_info"]])


'formulation/0/chemical/SMILES'

In [102]:
# Remove the original "uuid" field if you want to avoid duplication
if "uuid" in data:
    data['@id'] = data['uuid']
    del data['uuid']

# Add the context to your data
data['@context'] = context['@context']



# Convert to JSON-LD
jsonld_data = jsonld.compact(data, context)
jsonld_data_framed = jsonld.frame(jsonld_data, jsonld_frame)

print(json.dumps(jsonld_data, indent=2))

{
  "@context": {
    "ctime": "http://example.com/vocab/ctime",
    "formulation": "http://example.com/vocab/formulation",
    "chemical": "http://example.com/vocab/chemical",
    "SMILES": "http://example.com/vocab/SMILES",
    "InChIKey": "http://example.com/vocab/InChIKey",
    "fraction": "http://example.com/vocab/fraction",
    "fraction_type": "http://example.com/vocab/fraction_type",
    "internal_reference": "http://example.com/vocab/internal_reference",
    "conductivity": "http://example.com/vocab/conductivity",
    "values": "http://example.com/vocab/values",
    "temperature": "http://example.com/vocab/temperature",
    "quantity": "http://example.com/vocab/quantity",
    "method": "http://example.com/vocab/method",
    "parameters": "http://example.com/vocab/parameters",
    "molecular_dynamics": "http://example.com/vocab/molecular_dynamics"
  },
  "@id": "5362797a-e5d9-45ec-90a2-4f79be84d683",
  "ctime": "2023-07-26T11:22:20"
}


In [100]:
print(json.dumps(jsonld_data_framed, indent=2))

{
  "@context": {
    "ctime": "http://example.com/vocab/ctime",
    "result": "http://example.com/vocab/result",
    "conductivity": "http://example.com/vocab/conductivity",
    "values": "http://example.com/vocab/values",
    "temperature": "http://example.com/vocab/temperature",
    "formulation": "http://example.com/vocab/formulation",
    "chemical": "http://example.com/vocab/chemical",
    "SMILES": "http://example.com/vocab/SMILES",
    "InChIKey": "http://example.com/vocab/InChIKey"
  },
  "@id": "5362797a-e5d9-45ec-90a2-4f79be84d683",
  "ctime": "2023-07-26T11:22:20",
  "result": null
}


## Status
### 06.09.23
* Tried using the `frame` method of the JSON-LD specification. However, it filters keys from the original document, it cannot modify the original hierarchy. More specifically, if a level is not included in the frame, all the contents of that level dissapear. If the level is included, it inherits the original hierarchy.
* A more pragmatic solution could be:
    * Flattening the original json doc
    * Removing levels programatically, to simplify the original hierarchy
    * Re-nesting the simplified document
    * Attach a flat context to it.


> Next: Try if attaching a context to a normal json document, results in a file readable by rdflib